In [1]:
import matplotlib
matplotlib.use("nbagg")

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from utility import *
from models import *

In [2]:
import math

import torch
import torch.nn as nn
import torch.utils as utils
import torch.nn.init as init
import torch.utils.data as data
import torchvision.utils as v_utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import Tensor

import torch.optim as optim
import torch.nn.functional as F

In [3]:
from os import listdir
from os.path import isfile, join
import numpy as np

polycube_path = "/Users/davidcleres/DeepShape/Polycubing-Automated/Generated-Cars/"
polycube_files = [f for f in listdir(polycube_path) if isfile(join(polycube_path, f))]

voxelized_mesh_path = "/Users/davidcleres/DeepShape/Polycubing-Automated/voxelizedMeshes/"
voxelized_mesh_files = [f for f in listdir(voxelized_mesh_path) if isfile(join(voxelized_mesh_path, f))]

voxelizedFiles = []
polycubedFiles = []

for f in voxelized_mesh_files: 
    if f[-13:] == "voxelized.txt":
        voxelizedFiles = np.hstack((voxelizedFiles, f))
    
for f in polycube_files:
    if f[-14:] == "finalCubes.txt":
        polycubedFiles = np.hstack((polycubedFiles, f))

# Define the global parameters 

In [4]:
grid_size=32
batch_size=15

### Save the tensor to a text file 

In [5]:
voxelized_train_input, polycube_target=loadData(grid_size, polycube_path, voxelized_mesh_path, voxelizedFiles, polycubedFiles, loadFromScratch=True)

<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch.Size([32, 32, 32])
<class 'torch.IntTensor'> torch

# Load Data

In [6]:
preprocessed_input_train, preprocessed_input_validation, preprocessed_input_train_target, preprocessed_input_validation_target = preprocessing_train(voxelized_train_input, polycube_target,batch_size, False, False)

preprocessed_input_train = torch.from_numpy(preprocessed_input_train)
preprocessed_input_validation = torch.from_numpy(preprocessed_input_validation)
preprocessed_input_train_target = torch.from_numpy(preprocessed_input_train_target)
preprocessed_input_validation_target = torch.from_numpy(preprocessed_input_validation_target)

Ntrain = len(preprocessed_input_train[:, 0,0,0,0]) 
Nvalidation = len(preprocessed_input_validation[:,0,0,0,0])

train_input = Variable(preprocessed_input_train.view(Ntrain, 1, grid_size, grid_size, grid_size).float())
validation_input = Variable(preprocessed_input_validation.view(Nvalidation, 1, grid_size, grid_size, grid_size).float())

labels_train = preprocessed_input_train_target.float()
labels_validation = preprocessed_input_validation_target.float()

print('train', train_input.shape)
print('validation', validation_input.shape)
print('train_target', labels_train.shape)
print('validation_target', labels_validation.shape)

train torch.Size([45, 1, 32, 32, 32])
validation torch.Size([15, 1, 32, 32, 32])
train_target torch.Size([45, 32, 32, 32])
validation_target torch.Size([15, 32, 32, 32])


# Network initialization

In [ ]:
# Train network 
#criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()
#criterion = nn.PoissonNLLLoss()
#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.SmoothL1Loss() #interesting ... but does not converge
#criterion = nn.MSELoss() #0.83 but unstable

if isinstance(criterion, nn.CrossEntropyLoss):
    train_target = Variable(preprocessed_input_train_target).long()  # keep long tensors
    validation_target = Variable(preprocessed_input_validation_target, volatile=True).long() # convert to float
    Noutputs = 2
    
elif isinstance(criterion, nn.NLLLoss):
    train_target = Variable(preprocessed_input_train_target)  # keep long tensors
    validation_target = Variable(preprocessed_input_validation_target, volatile=True) # convert to float
    Noutputs = 2
    
else:
    train_target = Variable(preprocessed_input_train_target.float()) # convert to float
    validation_target = Variable(preprocessed_input_validation_target.float(), volatile=True ) # convert to float
    Noutputs = 1
    
Nbatches = int(math.ceil(Ntrain/batch_size)) #batch_size is defined above
Nepochs = 1000
Nrep = 1
        
#model = conv3DNet(grid_size, Noutputs, batch_size)
#model = conv3DNet(grid_size, Noutputs, batch_size)
#model = conv3DNet(grid_size, Noutputs, batch_size)
#model = conv3DNet(grid_size, Noutputs, batch_size)
model = UnetGenerator_3d(in_dim=1, out_dim=Noutputs, num_filter=4)

optimizer = optim.SGD(model.parameters(), lr=1e-1, momentum=0.90)
#optimizer = optim.Adam(model.parameters())
#optimizer = optim.Adagrad(model.parameters())
#optimizer = optim.Adamax(model.parameters())
#optimizer = optim.ASGD(model.parameters())
#optimizer = optim.RMSprop(model.parameters())
#optimizer = optim.Rprop(model.parameters())
 
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True) #Reduces the learning rate if it did not decreased by more than 10^-4 in 10 steps

train_errors = torch.Tensor(Nepochs).zero_()
validation_errors = torch.Tensor(Nepochs).zero_()

ep_loss = torch.Tensor(Nepochs).zero_()

for i_ep in range(Nepochs):
    for b_start in range(0, Ntrain, batch_size):
        bsize_eff = batch_size - max(0, b_start+batch_size-Ntrain)  # boundary case
        model.train()
        model.zero_grad()
        output = model(train_input.narrow(0, b_start, bsize_eff))
        if isinstance(criterion, nn.CrossEntropyLoss) or isinstance(criterion, nn.NLLLoss):
            batch_loss = criterion(output, train_target.narrow(0, b_start, bsize_eff))
        else:
            #if delta model is chosen
            #batch_loss = criterion(output.view(bsize_eff*Noutputs), train_target.narrow(0, b_start, bsize_eff))
            batch_loss = criterion(output.view(bsize_eff,grid_size,grid_size,grid_size), train_target.narrow(0, b_start, bsize_eff))
        ep_loss[i_ep] += batch_loss.data[0]
        batch_loss.backward()
        optimizer.step()

    scheduler.step(ep_loss[i_ep])

    nb_train_errs = compute_nb_errors(model, train_input, train_target, batch_size, criterion)
    nb_validation_errs = compute_nb_errors(model, validation_input, validation_target, batch_size, criterion)

    Ntrain_nb = Ntrain*grid_size**3
    Nvalidation_nb = Nvalidation*grid_size**3
    print("Epoch Number : ", i_ep)
    print("\t Training accuracy: ", (100*(Ntrain_nb-nb_train_errs)/Ntrain_nb))
    print("\t Validation accuracy ",(100*(Nvalidation_nb-nb_validation_errs)/Nvalidation_nb))

    print("\t Epoch Loss ", ep_loss[i_ep])

    train_errors[i_ep] = nb_train_errs
    validation_errors[i_ep] = nb_validation_errs


------Initiating U-Net------

Epoch Number :  0
	 Training accuracy:  64.96602376302083
	 Validation accuracy  65.4388427734375
	 Epoch Loss  2.0841779708862305
Epoch Number :  1
	 Training accuracy:  79.36876085069444
	 Validation accuracy  80.45918782552083
	 Epoch Loss  2.043081760406494
Epoch Number :  2
	 Training accuracy:  85.40181477864583
	 Validation accuracy  86.31917317708333
	 Epoch Loss  1.9918403625488281
Epoch Number :  3
	 Training accuracy:  86.71773274739583
	 Validation accuracy  87.52583821614583
	 Epoch Loss  1.9349089860916138
Epoch Number :  4
	 Training accuracy:  88.53719075520833
	 Validation accuracy  89.25435384114583
	 Epoch Loss  1.881852149963379
Epoch Number :  5
	 Training accuracy:  90.76571994357639
	 Validation accuracy  91.4459228515625
	 Epoch Loss  1.8280900716781616
Epoch Number :  6
	 Training accuracy:  92.5506591796875
	 Validation accuracy  93.24849446614583
	 Epoch Loss  1.7747764587402344
Epoch Number :  7
	 Training accuracy:  93.9375135

Epoch Number :  62
	 Training accuracy:  97.91978624131944
	 Validation accuracy  98.05419921875
	 Epoch Loss  1.307877540588379
Epoch Number :  63
	 Training accuracy:  97.86329481336806
	 Validation accuracy  97.9876708984375
	 Epoch Loss  1.3074092864990234
Epoch Number :  64
	 Training accuracy:  97.92826334635417
	 Validation accuracy  98.00394694010417
	 Epoch Loss  1.3065658807754517
Epoch Number :  65
	 Training accuracy:  97.98848470052083
	 Validation accuracy  97.989501953125
	 Epoch Loss  1.3049371242523193
Epoch Number :  66
	 Training accuracy:  97.93531629774306
	 Validation accuracy  98.08573404947917
	 Epoch Loss  1.3049880266189575
Epoch Number :  67
	 Training accuracy:  97.83426920572917
	 Validation accuracy  97.98929850260417
	 Epoch Loss  1.3031405210494995
Epoch Number :  68
	 Training accuracy:  98.02842881944444
	 Validation accuracy  98.03263346354167
	 Epoch Loss  1.300304651260376
Epoch Number :  69
	 Training accuracy:  97.90961371527777
	 Validation accur

Epoch Number :  124
	 Training accuracy:  98.72016059027777
	 Validation accuracy  98.31298828125
	 Epoch Loss  1.2015063762664795
Epoch Number :  125
	 Training accuracy:  98.80513509114583
	 Validation accuracy  98.32255045572917
	 Epoch Loss  1.198525071144104
Epoch Number :  126
	 Training accuracy:  98.97650824652777
	 Validation accuracy  98.29569498697917
	 Epoch Loss  1.1974866390228271
Epoch Number :  127
	 Training accuracy:  98.61436631944444
	 Validation accuracy  98.28267415364583
	 Epoch Loss  1.196487545967102
Epoch Number :  128
	 Training accuracy:  98.9666748046875
	 Validation accuracy  98.28084309895833
	 Epoch Loss  1.1956039667129517
Epoch Number :  129
	 Training accuracy:  98.78533257378473
	 Validation accuracy  98.31827799479167
	 Epoch Loss  1.193110466003418
Epoch Number :  130
	 Training accuracy:  98.82527669270833
	 Validation accuracy  98.4637451171875
	 Epoch Loss  1.1915693283081055
Epoch Number :  131
	 Training accuracy:  98.93900553385417
	 Validati

Epoch Number :  186
	 Training accuracy:  99.47367350260417
	 Validation accuracy  98.50056966145833
	 Epoch Loss  1.1128132343292236
Epoch Number :  187
	 Training accuracy:  99.68241373697917
	 Validation accuracy  98.1939697265625
	 Epoch Loss  1.1121227741241455
Epoch Number :  188
	 Training accuracy:  99.18436686197917
	 Validation accuracy  98.48409016927083
	 Epoch Loss  1.1115702390670776
Epoch Number :  189
	 Training accuracy:  99.56888834635417
	 Validation accuracy  98.20515950520833
	 Epoch Loss  1.1121652126312256
Epoch Number :  190
	 Training accuracy:  99.62002224392361
	 Validation accuracy  98.48307291666667
	 Epoch Loss  1.1123788356781006
Epoch Number :  191
	 Training accuracy:  99.47347005208333
	 Validation accuracy  98.47330729166667
	 Epoch Loss  1.1103061437606812
Epoch Number :  192
	 Training accuracy:  99.56169976128473
	 Validation accuracy  98.32478841145833
	 Epoch Loss  1.1086558103561401
Epoch Number :  193
	 Training accuracy:  99.46438259548611
	 V

Epoch Number :  248
	 Training accuracy:  99.94757758246527
	 Validation accuracy  98.31136067708333
	 Epoch Loss  1.0679733753204346
Epoch Number :  249
	 Training accuracy:  99.95517306857639
	 Validation accuracy  98.3441162109375
	 Epoch Loss  1.0671279430389404
Epoch Number :  250
	 Training accuracy:  99.95307074652777
	 Validation accuracy  98.363037109375
	 Epoch Loss  1.066465973854065
Epoch Number :  251
	 Training accuracy:  99.95659722222223
	 Validation accuracy  98.37381998697917
	 Epoch Loss  1.065845012664795
Epoch Number :  252
	 Training accuracy:  99.95686848958333
	 Validation accuracy  98.27596028645833
	 Epoch Loss  1.0654391050338745
Epoch Number :  253
	 Training accuracy:  99.9505615234375
	 Validation accuracy  98.4246826171875
	 Epoch Loss  1.064955472946167
Epoch Number :  254
	 Training accuracy:  99.95876736111111
	 Validation accuracy  98.22896321614583
	 Epoch Loss  1.0650272369384766
Epoch Number :  255
	 Training accuracy:  99.94357638888889
	 Validati

In [ ]:
print(train_target.shape)

In [ ]:
print(train_target[44,:,:,:])

#### 

In [ ]:
print(output.shape)

In [ ]:
print(output[14,1,:,:,:])

In [ ]:
training_accuracy = np.array(100*(Ntrain_nb-train_errors)/Ntrain_nb)
validation_accurcy = np.array(100*(Nvalidation_nb-validation_errors)/Nvalidation_nb)

In [ ]:
plt.plot(training_accuracy)
plt.plot(validation_accurcy)
plt.show()

In [ ]:
test_visualisation = output[14,1,:,:,:].round()

In [ ]:
voxels = np.array(test_visualisation.data)

# and plot everything
fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
ax.voxels(voxels)
fig.savefig('VoxelizedFinal.png')
fig.show()

In [ ]:
'''def summary(input_size, model):
        def register_hook(module):
            def hook(module, input, output):
                class_name = str(module.__class__).split('.')[-1].split("'")[0]
                module_idx = len(summary)

                m_key = '%s-%i' % (class_name, module_idx+1)
                summary[m_key] = OrderedDict()
                summary[m_key]['input_shape'] = list(input[0].size())
                summary[m_key]['input_shape'][0] = -1
                summary[m_key]['output_shape'] = list(output.size())
                summary[m_key]['output_shape'][0] = -1

                params = 0
                if hasattr(module, 'weight'):
                    params += th.prod(th.LongTensor(list(module.weight.size())))
                    if module.weight.requires_grad:
                        summary[m_key]['trainable'] = True
                    else:
                        summary[m_key]['trainable'] = False
                if hasattr(module, 'bias'):
                    params +=  th.prod(th.LongTensor(list(module.bias.size())))
                summary[m_key]['nb_params'] = params
                
            if not isinstance(module, nn.Sequential) and \
               not isinstance(module, nn.ModuleList) and \
               not (module == model):
                hooks.append(module.register_forward_hook(hook))
                
        dtype = th.cuda.FloatTensor
        
        # check if there are multiple inputs to the network
        if isinstance(input_size[0], (list, tuple)):
            x = [Variable(th.rand(1,*in_size)).type(dtype) for in_size in input_size]
        else:
            x = Variable(th.rand(1,*input_size)).type(dtype)
            
        print(x.shape)
        print(type(x[0]))
        # create properties
        summary = OrderedDict()
        hooks = []
        # register hook
        model.apply(register_hook)
        # make a forward pass
        model(x)
        # remove these hooks
        for h in hooks:
            h.remove()

        print('----------------------------------------------------------------')
        line_new = '{:>20}  {:>25} {:>15}'.format('Layer (type)', 'Output Shpae', 'Param #')
        print(line_new)
        print('================================================================')
        total_params = 0
        trainable_params = 0
        for layer in summary:
            ## input_shape, output_shape, trainable, nb_params
            line_new = '{:>20}  {:>25} {:>15}'.format(layer, summary[layer]['output_shape'], summary[layer]['nb_params'])
            total_params += summary[layer]['nb_params']
            if 'trainable' in summary[layer]:
                if summary[layer]['trainable'] == True:
                    trainable_params += summary[layer]['nb_params']
            print(line_new)
        print('================================================================')
        print('Total params: ' + str(total_params))
        print('Trainable params: ' + str(trainable_params))
        print('Non-trainable params: ' + str(total_params - trainable_params))
        print('----------------------------------------------------------------')
        return summary'''